# Question 1)

In [1]:
import numpy as np

In [2]:
class Dynamics:
    pass

In [3]:
hw2dynamics=Dynamics()
hw2dynamics.S0 = 100
hw2dynamics.r = 0.06
hw2dynamics.q = 0.01
hw2dynamics.maxvol = 0.6
hw2dynamics.localvol = lambda S,t: np.minimum(0.2+5*np.log(S/100)**2+0.1*np.exp(-t), 0.6)

# Note that hw2dynamics.localvol is a function 
# that may be invoked in the usual way, for example:
# hw2dynamics.localvol( exchangerate , time )

In [4]:
class Contract:
    pass

In [5]:
hw2contract=Contract()
hw2contract.putexpiry = 0.75;
hw2contract.putstrike = 95;
hw2contract.callexpiry = 0.25;
hw2contract.callstrike = 10;

In [6]:
class Tree:
    pass

In [7]:
hw2tree=Tree()
hw2tree.N=3000  #change if necessary to get $0.01 accuracy, in your judgment

In [8]:
# You complete the coding of this function

def compute_prob(r, sigma, deltat, deltax):
    nu =       r - (sigma**2)/2
    Pu =       0.5*(((sigma**2 * deltat) + nu**2*deltat**2)/(deltax**2)+(nu*deltat)/deltax)
    Pd =       0.5*(((sigma**2 * deltat) + nu**2*deltat**2)/(deltax**2)-(nu*deltat)/deltax)
    Pm =       1-Pu-Pd
    
    return Pu, Pm, Pd

def pricer_compound_localvol_trinom(contract,dynamics,tree):
    
    S0, r, q, localvol, maxvol = dynamics.S0, dynamics.r, dynamics.q, dynamics.localvol, dynamics.maxvol
    K_p, T_p, K_c, T_c = contract.putstrike, contract.putexpiry, contract.callstrike, contract.callexpiry
    N = tree.N
    
    avg_vol = np.minimum(0.2+5*(np.log(S0/100))**2+0.1,0.6)
    deltat =  T_p/N
    
    deltax = np.max([maxvol,np.sqrt(3)*avg_vol])  * np.sqrt(deltat)
    
    S=S0*np.exp(np.linspace(N, -N, num=2*N+1, endpoint=True)*deltax) 
    
    optionprice_p = np.maximum(K_p-S,0)   #an array of time-T option prices
    
    # Computing Price American Put and Compound Call
    call_flag = True 
    for k in range(N-1,-1,-1):
        # Time point
        t = k*deltat
        # Stock prices for all the nodes at that timestamp 
        S=S0*np.exp(np.linspace(k, -k, num=2*k+1, endpoint=True)*deltax)
        # Local sigma for all nodes
        sigma = localvol(S,t)
        # Local probabilities for all nodes 
        pu, pm, pd = compute_prob(r-q, sigma, deltat, deltax)
        if k == (N-1):
            print(pu)
            print(pm)
            print(pd)
        # Check if we are now at T=0.25 or the first timestamp in the grid before that for pricing the call option 
        if t <= T_c and call_flag is True:
            #Price of the option at the terminal node is equal the price of the put for each of those nodes minus the strike
            optionprice_c = np.maximum(optionprice_p-K_c,0)
            call_flag = False
                
        for j in range(2*k-1):
            up = optionprice_p[j]
            mid = optionprice_p[j+1]
            down = optionprice_p[j+2]
            discount = np.exp(-r*deltat)
            # Price of American Put is the max between the discounted expected value next time or the intrinsic value of the option 
            optionprice_p[j] = np.max([discount*(pu[j]*up + pm[j]*mid + pd[j]*down), K_p - S[j]])
            
            if t < T_c:
                up = optionprice_c[j]
                mid = optionprice_c[j+1]
                down = optionprice_c[j+2]
                discount = np.exp(-r*deltat)
                # European Option discounted expected value of the next time 
                optionprice_c[j] = discount*(pu[j]*up + pm[j]*mid + pd[j]*down)
                                        
    price_of_put = optionprice_p[0]
    
    price_of_call_on_put = optionprice_c[0] 
    
    return (price_of_put, price_of_call_on_put)

In [ ]:
(answer_part_a, answer_part_b) = pricer_compound_localvol_trinom(hw2contract,hw2dynamics,hw2tree)
print("Time-0 price of an American-style put on the GDF with strike 95 and expires at time 0.75 is equal to: {:.4f}".format(answer_part_a))
print("Time-0 price of a European-style call on the GDF put with strike 10 and expires at time 0.25 is equal to: {:.4f}".format(answer_part_b))

[0.49829297 0.49829297 0.49829297 ... 0.49829297 0.49829297 0.49829297]
[-1.17361111e-05 -1.17361111e-05 -1.17361111e-05 ... -1.17361111e-05
 -1.17361111e-05 -1.17361111e-05]
[0.50171877 0.50171877 0.50171877 ... 0.50171877 0.50171877 0.50171877]


# Question 2)

## a)
$$ C = S_0N(d_1) - Ke^{rT}N(d_2)$$
$$ d_1 = \frac{\ln{\frac{S_0}{K}}+\left(r + \frac{\sigma^2}{2}\right)T}{\sigma\sqrt{T}} \quad d_2 = d_1 - \sigma\sqrt{T}$$

Given that $\ln{1} = 0$

$$ d_1 = \frac{\left(r + \frac{\sigma^2}{2}\right)\sqrt{T}}{\sigma} \quad d_2 = \frac{\left(r - \frac{\sigma^2}{2}\right)\sqrt{T}}{\sigma}$$

For small $x$ we can use the Taylor Approximation up to the first order of $N(x)$ where $N(x)$ is the standard normally distributed function.

$$ N(x) \approx N(0) + N'(0)x \quad N'(x) = \frac{1}{\sqrt{2\pi}}e^{-x^2/2} \rightarrow N'(0) = \frac{1}{\sqrt{2\pi}}$$

$$ N(x) \approx \frac{1}{2} + \frac{1}{\sqrt{2\pi}}x $$

Which leads us to 

$$ C = S_0 N\left(\frac{\left(r + \frac{\sigma^2}{2}\right)\sqrt{T}}{\sigma}\right) -Ke^{-rT}N\left(\frac{\left(r - \frac{\sigma^2}{2}\right)\sqrt{T}}{\sigma}\right) $$

$$ \Delta = \frac{\partial C}{\partial S} = N\left(\frac{\left(r + \frac{\sigma^2}{2}\right)\sqrt{T}}{\sigma}\right) \approx \frac{1}{2} + \frac{1}{\sqrt{2\pi}}\left(\frac{\left(r + \frac{\sigma^2}{2}\right)\sqrt{T}}{\sigma}\right) $$
Plug $\sigma = 0.2, T= 0.25, r = 0.01$

$$ \Delta = \frac{1}{2} + \frac{1}{\sqrt{2\pi}}\left(\frac{\left(0.01 + \frac{0.2^2}{2}\right)\sqrt{0.25}}{0.2}\right) = 0.53 $$

## b)

Using 2nd-Order Taylor Approximation to $c(x)$

$$ c(x) \approx c(0) + c'(0)\cdot x + \frac{1}{2}c''(0)\cdot x^2 $$
$$ c(x) \approx C(S_0) + \frac{\partial c}{\partial x}(0)\cdot x + \frac{1}{2}\frac{\partial^2 c}{\partial x^2}(0)\cdot x^2 $$
$$ c(x) \approx C(S_0) + 3x + 3x^2 $$

Given that we are pricing a contract at $S_0 = 3.6 \rightarrow x = -0.1$

$$ c(-0.1) \approx 2 + 3(-0.1) + 3(-0.1)^2 $$
$$ C(3.6) = c(-0.1) \approx 1.73 $$